In [1]:
import vrep
import time
import numpy as np

from scipy import linalg
from random import *
import random
from scipy.linalg import logm, expm, norm, inv


def skew(s):
             
                return np.array([[0 ,-s[2] ,s[1]], [s[2], 0 ,-s[0]],[-s[1] ,s[0], 0]])
        
def skew_S(s):
                A = np.array([[0 ,-s[0][2] ,s[0][1], s[0][3]], [s[0][2], 0 ,-s[0][0], s[0][4]],[-s[0][1] ,s[0][0], 0, s[0][5]],[0,0,0,0]])
                
                return A
            
def invert(T):
                
                R=T[[0,1,2],:][:,[0,1,2]]
                
                A = np.block([[R.transpose(), (-1)*(R.transpose()).dot(T[[0,1,2],:][:,[3]])],   [0,0,0,1]    ])

                return A
            
def Adj(T):
    
                A = np.block([[T[[0,1,2],:][:,[0,1,2]], np.zeros([3,3])]  ,  [ (skew(T[[0,1,2],:][:,[3]])).dot(T[[0,1,2],:][:,[0,1,2]]),  T[[0,1,2],:][:,[0,1,2]]  ]   ])
                
                return A
            

In [2]:
def build_path(T_f=None, T_b=None):
    q = mcat([])
    node_cur = T_f(size(T_f, 2))
    while (node_cur.parent != 0):
        q = mcat([node_cur.theta, q])
        node_cur = T_f(node_cur.parent)
     
    node_cur = T_b(1, size(T_b, 2))
    node_cur = T_b(node_cur.parent)
        
    while (node_cur.parent != 0):
        q = mcat([q, node_cur.theta])
        node_cur = T_b(node_cur.parent)
            
    q = mcat([T_f(1).theta, q, T_b(1).theta])
    
def find_parent(T=None, theta_rand=None):
    theta_best = T(1).theta
    theta_best_idx = 1
    dist_best = norm(theta_rand - theta_best)
    for i in mslice[1:size(T, 2)]:
        T_i = T(i)
        theta_i = T_i.theta
        dist = norm(theta_rand - theta_i)
        if dist < dist_best:
            dist_best = dist
            theta_best_idx = i
            end
            end
            close_idx = theta_best_idx
            end
           

In [3]:
def IsTwoBallCollision(p1,p2,r1,r2):
    #print('check6')
    collision_2balls = 0
    diff = np.array([0.0,0.0,0.0])
    diff[0] = p1[0] - p2[0]
    diff[1] = p1[1] - p2[1]
    diff[2] = p1[2] - p2[2]
    if np.linalg.norm(diff) < (r1 + r2):
        #print('check7')
        collision_2balls = 1
    return collision_2balls

def CollisionCheck(p_robot,p_obstacles):
    collision_check = 0
    #print('check3')
    for i in range(0,7):
        for j in range(0,7):
            if (IsTwoBallCollision(p_robot[[0,1,2],i],p_robot[[0,1,2],j],r_robot[i],r_robot[j]) == 1) and (i != j):
                collision_check = 1
                #print('check4')
                break
    for i in range(0,7):
        for j in range(0,2):
            if (IsTwoBallCollision(p_robot[[0,1,2],i],p_obstacles[[0,1,2],j],r_robot[i],r_obstacles[j]) == 1):
                collision_check = 1
                #print('check5')
                break  
                
                
    return collision_check



In [4]:
def CalculateSpheresPositions(theta_vec,q):
    q1=np.array([0,0,0.109])
    q2=np.array([-0.12,0,0.109])
    q3=np.array([-0.12,0,0.353])
    q4=np.array([-0.027,0,0.566])
    q5=np.array([-0.11,0,0.566])
    q6=np.array([-0.11,0,0.649])
    q7=np.array([-0.192,0,0.649])  #end effector
    
    q = np.block([[q1],[q2],[q3],[q4],[q5],[q6],[q7]])
    q = q.T
    
    result, obs1_pos=vrep.simxGetObjectPosition(clientID,sphere_obs1_handle,robot_handle,vrep.simx_opmode_blocking)
    if result != vrep.simx_return_ok:
        raise Exception('could not get current frame current position')

    result, obs2_pos=vrep.simxGetObjectPosition(clientID,sphere_obs2_handle,robot_handle,vrep.simx_opmode_blocking)
    if result != vrep.simx_return_ok:
        raise Exception('could not get current frame current position')

    p8 = obs1_pos
    p9 = obs2_pos

    extra1 = [-0.027,0,0.353]
    a1 = [0,0,1]
    a2 = [-1,0,0]
    a3 = [-1,0,0]
    a4 = [-1,0,0]
    a5 = [0,0,1]
    a6 = [-1,0,0]

    S1 = [a1,-np.dot(skew(a1),q1)]
    S1 = np.reshape(S1,(1,6))
    S2 = [a2,-np.dot(skew(a2),q2)]
    S2 = np.reshape(S2,(1,6))
    S3 = [a3,-np.dot(skew(a3),q3)]
    S3 = np.reshape(S3,(1,6))
    S4 = [a4,-np.dot(skew(a4),q4)]
    S4 = np.reshape(S4,(1,6))
    S5 = [a5,-np.dot(skew(a5),q5)]
    S5 = np.reshape(S5,(1,6))
    S6 = [a6,-np.dot(skew(a6),q6)]
    S6 = np.reshape(S6,(1,6))
    
    
    final_pj2=expm(skew_S(S1)*theta_vec[0]).dot(np.concatenate([q[:,1],[1]]))
    final_pj3=expm(skew_S(S1)*theta_vec[0]).dot(expm(skew_S(S2)*theta_vec[1])).dot(np.concatenate([q[:,2],[1]]))
    final_pextra1=expm(skew_S(S1)*theta_vec[0]).dot(expm(skew_S(S2)*theta_vec[1])).dot(np.concatenate([extra1,[1]]))
    final_pj4=expm(skew_S(S1)*theta_vec[0]).dot(expm(skew_S(S2)*theta_vec[1])).dot(expm(skew_S(S3)*theta_vec[2])).dot(np.concatenate([q[:,3],[1]]))
    final_pj5=expm(skew_S(S1)*theta_vec[0]).dot(expm(skew_S(S2)*theta_vec[1])).dot(expm(skew_S(S3)*theta_vec[2])).dot(expm(skew_S(S4)*theta_vec[3])).dot(np.concatenate([q[:,4],[1]]))
    final_pj6=expm(skew_S(S1)*theta_vec[0]).dot(expm(skew_S(S2)*theta_vec[1])).dot(expm(skew_S(S3)*theta_vec[2])).dot(expm(skew_S(S4)*theta_vec[3])).dot(expm(skew_S(S5)*theta_vec[4])).dot(np.concatenate([q[:,5],[1]]))
    final_pj7=expm(skew_S(S1)*theta_vec[0]).dot(expm(skew_S(S2)*theta_vec[1])).dot(expm(skew_S(S3)*theta_vec[2])).dot(expm(skew_S(S4)*theta_vec[3])).dot(expm(skew_S(S5)*theta_vec[4])).dot(expm(skew_S(S6)*theta_vec[5])).dot(np.concatenate([q[:,6],[1]]))

    p1 = q1;
    p2 = final_pj2[0:3]
    p3 = final_pj3[0:3]
    pextra1 = final_pextra1[0:3]
    p4 = final_pj4[0:3]
    p5 = final_pj5[0:3]
    p6 = final_pj6[0:3]
    p7 = final_pj7[0:3]

    p_robot = np.concatenate(([p1],[p2],[p3],[pextra1],[p4],[p5],[p6],[p7]))
    p_robot = p_robot.T
    p_obstacles = np.concatenate(([p8],[p9]))
    p_obstacles = p_obstacles.T
    
   # print(p_robot)   
    return p_robot, p_obstacles 

In [5]:
def brack6(a):
    ws = a[:3]
    vs = a[3:]
    wb = skew(ws)
    ans = np.array([[wb[0,0], wb[0,1], wb[0,2], vs[0]],[wb[1,0], wb[1,1], wb[1,2], vs[1]],[wb[2,0], wb[2,1], wb[2,2], vs[2]], [0,0,0,0]])
    return ans
def revolute(a,q):
    ab = skew(a)
    bottom = -1*np.dot(ab,q)
    return np.array([[a[0]],[a[1]],[a[2]],[bottom[0]],[bottom[1]],[bottom[2]]])
def skew(m):
    return np.array([[0 ,-m[2], m[1]],[m[2], 0, -m[0]], [-m[1], m[0], 0]])

def inverse_kinematics(T):
    q1=np.array([0,0,0.109])
    q2=np.array([-0.12,0,0.109])
    q3=np.array([-0.12,0,0.353])
    q4=np.array([-0.027,0,0.566])
    q5=np.array([-0.11,0,0.566])
    q6=np.array([-0.11,0,0.649])
    q7=np.array([-0.192,0,0.649]) 
    
    a1 = [0,0,1]
    a2 = [-1,0,0]
    a3 = [-1,0,0]
    a4 = [-1,0,0]
    a5 = [0,0,1]
    a6 = [-1,0,0]
    
    
    M =[[0,0,-1,q7[0]],[0,1,0,q7[1]],[1,0,0,q7[2]],[0,0,0,1]]
    
    axis = [a1,a2,a3,a4,a5,a6]
    point = [q1,q2,q3,q4,q5,q6]
    
    screw = [revolute(axis[a],point[a]) for a in range(len(axis))]
    
    theta = np.repeat(np.pi/8,6).reshape(6,1)
    tolerance = 0.01
    error = 20
    num_iterations = 0
    while (error > tolerance) and (num_iterations < 500):
        exp = [linalg.expm(brack6(screw[s])*theta[s]) for s in range(len(screw))]
        Tc = np.identity(4)
        for i in exp:
            Tc = Tc.dot(i)
        Tc = Tc.dot(M)
        vb = linalg.logm(T.dot(np.linalg.inv(Tc)))
        v = np.array([vb[2,1],vb[0,2],vb[1,0],vb[0,3],vb[1,3],vb[2,3]]).reshape(6,1)
        
        jacobian = []
        jacobian.append(screw[0])
        j = np.identity(4)
        for i in range(len(exp)-1):
            j = j.dot(exp[i])
            jacobian.append(Adj(j).dot(screw[i+1]))
        
        space = np.hstack((i for i in jacobian))
        
        thetadot = (np.linalg.inv(np.add(np.transpose(space).dot(space),0.1*np.eye(6)))).dot(np.transpose(space).dot(v))
        theta = np.add(theta,thetadot)
        n = np.linalg.norm(v)
        error = abs(n)
        num_iterations+=1
    print ("Iterations: ",num_iterations)
    if (num_iterations == 500):
        return None
    return theta


In [6]:
# Close all open connections (just in case)
vrep.simxFinish(-1)

# Connect to V-REP (raise exception on failure)
clientID = vrep.simxStart('127.0.0.1', 19997, True, True, 5000, 5)
if clientID == -1:
    raise Exception('Failed connecting to remote API server')

In [7]:
#HANDLES FOR JOINTS, DUMMIES AND OBSTACLES

# Get "handle" to the first joint of robot
result, joint_one_handle = vrep.simxGetObjectHandle(clientID, 'UR3_joint1', vrep.simx_opmode_blocking)
if result != vrep.simx_return_ok:
    raise Exception('could not get object handle for first joint')

# Get "handle" to the second joint of robot
result, joint_two_handle = vrep.simxGetObjectHandle(clientID, 'UR3_joint2', vrep.simx_opmode_blocking)
if result != vrep.simx_return_ok:
    raise Exception('could not get object handle for second joint')
    
# Get "handle" to the third joint of robot
result, joint_three_handle = vrep.simxGetObjectHandle(clientID, 'UR3_joint3', vrep.simx_opmode_blocking)
if result != vrep.simx_return_ok:
    raise Exception('could not get object handle for third joint')
    
# Get "handle" to the fourth joint of robot
result, joint_four_handle = vrep.simxGetObjectHandle(clientID, 'UR3_joint4', vrep.simx_opmode_blocking)
if result != vrep.simx_return_ok:
    raise Exception('could not get object handle for fourth joint')
    
# Get "handle" to the fifth joint of robot
result, joint_five_handle = vrep.simxGetObjectHandle(clientID, 'UR3_joint5', vrep.simx_opmode_blocking)
if result != vrep.simx_return_ok:
    raise Exception('could not get object handle for first joint')
    
# Get "handle" to the sixth joint of robot
result, joint_six_handle = vrep.simxGetObjectHandle(clientID, 'UR3_joint6', vrep.simx_opmode_blocking)
if result != vrep.simx_return_ok:
    raise Exception('could not get object handle for first joint')
    
# Get "handle" to the robot
result, robot_handle = vrep.simxGetObjectHandle(clientID, 'UR3_link1_visible', vrep.simx_opmode_blocking)
if result != vrep.simx_return_ok:
    raise Exception('could not get object handle for robot')
    
    
    
# SPHERES IN THE ROBOT

result, sphere_joint1_handle = vrep.simxGetObjectHandle(clientID,'Dummy', vrep.simx_opmode_blocking)
if result != vrep.simx_return_ok:
    raise Exception('could not get object handle for frame')

result, sphere_joint2_handle = vrep.simxGetObjectHandle(clientID,'Dummy0', vrep.simx_opmode_blocking)
if result != vrep.simx_return_ok:
    raise Exception('could not get object handle for frame')

result, sphere_joint3_handle = vrep.simxGetObjectHandle(clientID,'Dummy1', vrep.simx_opmode_blocking)
if result != vrep.simx_return_ok:
    raise Exception('could not get object handle for frame')

result, sphere_extra1_handle = vrep.simxGetObjectHandle(clientID,'Dummy2', vrep.simx_opmode_blocking)
if result != vrep.simx_return_ok:
    raise Exception('could not get object handle for frame')

result, sphere_joint4_handle = vrep.simxGetObjectHandle(clientID,'Dummy3', vrep.simx_opmode_blocking)
if result != vrep.simx_return_ok:
    raise Exception('could not get object handle for frame')

result, sphere_joint5_handle = vrep.simxGetObjectHandle(clientID,'Dummy4', vrep.simx_opmode_blocking)
if result != vrep.simx_return_ok:
    raise Exception('could not get object handle for frame')
    
result, sphere_joint6_handle = vrep.simxGetObjectHandle(clientID,'Dummy5', vrep.simx_opmode_blocking)
if result != vrep.simx_return_ok:
    raise Exception('could not get object handle for frame')

result, sphere_end_effector_handle = vrep.simxGetObjectHandle(clientID,'Dummy6', vrep.simx_opmode_blocking)
if result != vrep.simx_return_ok:
    raise Exception('could not get object handle for frame')
    

# OBSTACLES

result, sphere_obs1_handle = vrep.simxGetObjectHandle(clientID,'Dummy7', vrep.simx_opmode_blocking)
if result != vrep.simx_return_ok:
    raise Exception('could not get object handle for frame')

result, sphere_obs2_handle = vrep.simxGetObjectHandle(clientID,'Dummy8', vrep.simx_opmode_blocking)
if result != vrep.simx_return_ok:
    raise Exception('could not get object handle for frame')


In [8]:
# Start simulation
vrep.simxStartSimulation(clientID, vrep.simx_opmode_oneshot)

1

In [9]:
#SPHERES TO INITIAL POSITION

#Initial position 
q1=np.array([0,0,0.109])
q2=np.array([-0.12,0,0.109])
q3=np.array([-0.12,0,0.353])
q4=np.array([-0.027,0,0.566])
q5=np.array([-0.11,0,0.566])
q6=np.array([-0.11,0,0.649])
q7=np.array([-0.192,0,0.649])  #end effector


q = np.block([[q1],[q2],[q3],[q4],[q5],[q6],[q7]])
q = q.T

extra1 = [-0.027,0,0.353]

a1 = [0,0,1]
a2 = [-1,0,0]
a3 = [-1,0,0]
a4 = [-1,0,0]
a5 = [0,0,1]
a6 = [-1,0,0]

S1 = [a1,-np.dot(skew(a1),q1.T)]
S1 = np.reshape(S1,(1,6))
S2 = [a2,-np.dot(skew(a2),q2)]
S2 = np.reshape(S2,(1,6))
S3 = [a3,-np.dot(skew(a3),q3)]
S3 = np.reshape(S3,(1,6))
S4 = [a4,-np.dot(skew(a4),q4)]
S4 = np.reshape(S4,(1,6))
S5 = [a5,-np.dot(skew(a5),q5)]
S5 = np.reshape(S5,(1,6))
S6 = [a6,-np.dot(skew(a6),q6)]
S6 = np.reshape(S6,(1,6))


S = np.concatenate((S1,S2,S3,S4,S5,S6))
S = S.T

r_robot = (0.05,0.05,0.025,0.025,0.0125,0.0125,0.0125)
r_obstacles = (0.5,0.25)

result = vrep.simxSetObjectPosition(clientID,sphere_joint1_handle,robot_handle,q1,vrep.simx_opmode_blocking)
if result != vrep.simx_return_ok:
    raise Exception('could not set frame new position')
    
result = vrep.simxSetObjectPosition(clientID,sphere_joint2_handle,robot_handle,q2,vrep.simx_opmode_blocking)
if result != vrep.simx_return_ok:
    raise Exception('could not set frame new position')

result = vrep.simxSetObjectPosition(clientID,sphere_joint3_handle,robot_handle,q3,vrep.simx_opmode_blocking)
if result != vrep.simx_return_ok:
    raise Exception('could not set frame new position')
    
result = vrep.simxSetObjectPosition(clientID,sphere_extra1_handle,robot_handle,extra1,vrep.simx_opmode_blocking)
if result != vrep.simx_return_ok:
    raise Exception('could not set frame new position')
    
result = vrep.simxSetObjectPosition(clientID,sphere_joint4_handle,robot_handle,q4,vrep.simx_opmode_blocking)
if result != vrep.simx_return_ok:
    raise Exception('could not set frame new position')
    
result = vrep.simxSetObjectPosition(clientID,sphere_joint5_handle,robot_handle,q5,vrep.simx_opmode_blocking)
if result != vrep.simx_return_ok:
    raise Exception('could not set frame new position')
    
result = vrep.simxSetObjectPosition(clientID,sphere_joint6_handle,robot_handle,q6,vrep.simx_opmode_blocking)
if result != vrep.simx_return_ok:
    raise Exception('could not set frame new position')

result = vrep.simxSetObjectPosition(clientID,sphere_end_effector_handle,robot_handle,q7,vrep.simx_opmode_blocking)
if result != vrep.simx_return_ok:
    raise Exception('could not set frame new position')


In [10]:
#FIRST JOINT

# Get the current value of the first joint variable
result, theta = vrep.simxGetJointPosition(clientID, joint_one_handle, vrep.simx_opmode_blocking)
if result != vrep.simx_return_ok:
    raise Exception('could not get first joint variable')
print('current value of first joint variable: theta = {:f}'.format(theta))

pret, rjoint1 = vrep.simxGetObjectPosition(clientID, joint_one_handle,-1, vrep.simx_opmode_oneshot_wait)
print('The position of the first joint is {} '.format( rjoint1))


#SECOND JOINT


# Get the current value of the second joint variable
result, theta = vrep.simxGetJointPosition(clientID, joint_two_handle, vrep.simx_opmode_blocking)
if result != vrep.simx_return_ok:
    raise Exception('could not get second joint variable')
print('current value of second joint variable: theta = {:f}'.format(theta))

pret, rjoint2 = vrep.simxGetObjectPosition(clientID, joint_two_handle,-1, vrep.simx_opmode_oneshot_wait)
print('The position of the second joint is {} '.format( rjoint2))


# THIRD JOINT

# Get the current value of the third joint variable
result, theta = vrep.simxGetJointPosition(clientID, joint_three_handle, vrep.simx_opmode_blocking)
if result != vrep.simx_return_ok:
    raise Exception('could not get third joint variable')
print('current value of third joint variable: theta = {:f}'.format(theta))

pret, rjoint3 = vrep.simxGetObjectPosition(clientID, joint_three_handle,-1, vrep.simx_opmode_oneshot_wait)
print('The position of the third joint is {} '.format( rjoint3))



#FOURTH JOINT

# Get the current value of the fourth joint variable
result, theta = vrep.simxGetJointPosition(clientID, joint_four_handle, vrep.simx_opmode_blocking)
if result != vrep.simx_return_ok:
    raise Exception('could not get fourth joint variable')
print('current value of fourth joint variable: theta = {:f}'.format(theta))

pret, rjoint4 = vrep.simxGetObjectPosition(clientID, joint_four_handle,-1, vrep.simx_opmode_oneshot_wait)
print('The position of the fourth joint is {} '.format( rjoint4))


#FIFTH JOINT

# Get the current value of the fifth joint variable
result, theta = vrep.simxGetJointPosition(clientID, joint_five_handle, vrep.simx_opmode_blocking)
if result != vrep.simx_return_ok:
    raise Exception('could not get fifth joint variable')
print('current value of fifth joint variable: theta = {:f}'.format(theta))

pret, rjoint5 = vrep.simxGetObjectPosition(clientID, joint_five_handle,-1, vrep.simx_opmode_oneshot_wait)
print('The position of the fifth joint is {} '.format( rjoint5))


#SIXTH JOINT


# Get the current value of the sixth joint variable
result, theta = vrep.simxGetJointPosition(clientID, joint_six_handle, vrep.simx_opmode_blocking)
if result != vrep.simx_return_ok:
    raise Exception('could not get sixth joint variable')
print('current value of sixth joint variable: theta = {:f}'.format(theta))


pret, rjoint6 = vrep.simxGetObjectPosition(clientID, joint_six_handle,-1, vrep.simx_opmode_oneshot_wait)
print('The position of the sixth joint is {} '.format( rjoint6))



#END - EFFECTOR

result, end_effector_handle = vrep.simxGetObjectHandle(clientID,'UR3_connection', vrep.simx_opmode_blocking)
if result != vrep.simx_return_ok:
    raise Exception('could not get object handle for frame')

result, end_effector_pos=vrep.simxGetObjectPosition(clientID,end_effector_handle,joint_one_handle,vrep.simx_opmode_blocking)
if result != vrep.simx_return_ok:
    raise Exception('could not get current frame current position')

current value of first joint variable: theta = 0.754738
The position of the first joint is [-0.44999998807907104, 1.0749999284744263, 0.14749836921691895] 
current value of second joint variable: theta = 0.826094
The position of the second joint is [-0.5163426399230957, 0.9851514101028442, 0.1518017053604126] 
current value of third joint variable: theta = -0.980325
The position of the third joint is [-0.6912535429000854, 1.0775107145309448, 0.26866400241851807] 
current value of fourth joint variable: theta = 1.188653
The position of the fourth joint is [-0.7016915082931519, 1.0559953451156616, 0.45624926686286926] 
current value of fifth joint variable: theta = 1.265951
The position of the fifth joint is [-0.7859800457954407, 1.0453273057937622, 0.44622647762298584] 
current value of sixth joint variable: theta = 1.347979
The position of the sixth joint is [-0.7890530824661255, 1.019516110420227, 0.42002561688423157] 


In [11]:
#DEFINE/CREATE T_1in0 HERE

T_1in0 = np.array([[-0.03957477, -0.96134411,  0.27249099,  0.25685692],
      [ 0.63132818,  0.18732024,  0.7525529 ,  0.25710661],
      [-0.77450537,  0.20181335,  0.59951047,  0.3646851 ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

theta_g = inverse_kinematics(T_1in0)
theta_goal=np.array([theta_g[0][0],theta_g[1][0],theta_g[2][0],theta_g[3][0],theta_g[4][0],theta_g[5][0]])
print(theta_goal)

Iterations:  31
[-1.22562556  0.06633541  1.40178655  0.81004128  0.90588046  0.87950189]


In [12]:
frame_target_pos = [T_1in0[0][3],T_1in0[1][3],T_1in0[2][3]]
print('Target position:', frame_target_pos)

result, frame_handle = vrep.simxGetObjectHandle(clientID,'ReferenceFrame', vrep.simx_opmode_blocking)
if result != vrep.simx_return_ok:
    raise Exception('could not get object handle for frame')

result, current_frame_pos=vrep.simxGetObjectPosition(clientID,frame_handle,robot_handle,vrep.simx_opmode_blocking)
if result != vrep.simx_return_ok:
    raise Exception('could not get current frame current position')
    
print('Position before change:', current_frame_pos)   

result = vrep.simxSetObjectPosition(clientID,frame_handle,robot_handle,frame_target_pos,vrep.simx_opmode_blocking)
if result != vrep.simx_return_ok:
    raise Exception('could not set frame new position')
    
    
result, current_frame_pos=vrep.simxGetObjectPosition(clientID,frame_handle,robot_handle,vrep.simx_opmode_blocking)
if result != vrep.simx_return_ok:
    raise Exception('could not get current frame current position')
print('Final position:', current_frame_pos) 


Target position: [0.25685692, 0.25710661, 0.3646851]
Position before change: [-1.375, 0.7499998807907104, -0.04302540421485901]
Final position: [0.25685691833496094, 0.2571065425872803, 0.3646850883960724]


In [13]:
def collision_path(S,M, p_robot, r_robot, p_obstacle, r_obstacles, theta_start, theta_goal):
    col = 0
    #print('check1')
    for m in range(1,100):
        t=(1-m/1000)*theta_start + m/1000*theta_goal
        p_robot, p_obstacle = CalculateSpheresPositions(t,p_robot)
        CollisionCheck(p_robot,p_obstacle)
        if CollisionCheck == 1:
            col = 1
            #print('check2')
            break
    return col

In [14]:
#FIND PATH
#Initializing the tree
class Tree_Node:
    def __init__(self, value, parent_index):
        self.value = value
        self.parent_index = parent_index
        
q1=np.array([0,0,0.109])
q2=np.array([-0.12,0,0.109])
q3=np.array([-0.12,0,0.353])
q4=np.array([-0.027,0,0.566])
q5=np.array([-0.11,0,0.566])
q6=np.array([-0.11,0,0.649])
q7=np.array([-0.192,0,0.649])  #end effector

q = np.concatenate((q1,q2.T,q3.T,q4.T,q5.T,q6.T))
q = q.T

M =[[0,0,-1,q7[0]],[0,1,0,q7[1]],[1,0,0,q7[2]],[0,0,0,1]]
        
theta_start = np.array([0.1,0.1,0.1,0.1,0.1,0.1])
theta_path_start = Tree_Node(theta_start,0)
p_robot, p_obstacle = CalculateSpheresPositions(theta_path_start.value,q)
#print(theta_path_start.value)
i=0
last_node = Tree_Node(theta_start,1)
Joint_angles=np.zeros((6,15)) #6 rows 15 columns
Joint_angles[:,i]=theta_start
#print(Joint_angles)

while(collision_path(S,M,p_robot, r_robot, p_obstacle, r_obstacles, last_node.value, theta_goal)==1):
    theta = np.random.uniform(size(0.1,0.3,6))
    print(theta)
    if(collision_path(S,M,p_robot,r_robot,p_obstacle,r_obstacle,last_node.value,theta)==0):
        last_node.value=theta
        p_robot, p_obstacle = CalculateSpheresPositions(last_node.value,q)
        i=i+1
        Joint_angles[:,i]=theta
        last_node.parent_index=i
        print(i)
        if(collision_path(S,M,p_robot,r_robot,p_obstacle,r_obstacle,theta,theta_goal)==0):
            break
            
Joint_angles[:,last_node.parent_index]=theta_goal
#Joint_angles[:,last_node.parent_index]=([-1.24278829 , 1.36920013, -1.45335425  ,2.34078196,  0.89940637  ,0.89906668])
#Joint_angles[:,last_node.parent_index]=([-1.22562556, 0.06633541, 1.40178655, 0.81004128, 0.90588046, 0.87950189])


#MOVE ACCORDING TO THE DIFFERENT SETS OF THETAS
print(Joint_angles)
#print(Joint_angles[:,2])

#print(p_robot)
#print(p_robot[[0,1,2],0])

for l in range(0,last_node.parent_index+1):
    #print(Joint_angles[:,l])
    next_theta = Joint_angles[:,l]

    vrep.simxSetJointTargetPosition(clientID, joint_one_handle, next_theta[0], vrep.simx_opmode_oneshot)
    vrep.simxSetObjectPosition(clientID,sphere_joint1_handle,robot_handle,p_robot[[0,1,2],0],vrep.simx_opmode_blocking)

    vrep.simxSetJointTargetPosition(clientID, joint_two_handle, next_theta[1], vrep.simx_opmode_oneshot)
    vrep.simxSetObjectPosition(clientID,sphere_joint2_handle,robot_handle,p_robot[[0,1,2],1],vrep.simx_opmode_blocking)

    vrep.simxSetJointTargetPosition(clientID, joint_three_handle, next_theta[2], vrep.simx_opmode_oneshot)
    vrep.simxSetObjectPosition(clientID,sphere_joint3_handle,robot_handle,p_robot[[0,1,2],2],vrep.simx_opmode_blocking)

    vrep.simxSetObjectPosition(clientID,sphere_extra1_handle,robot_handle,p_robot[[0,1,2],3],vrep.simx_opmode_blocking)

    vrep.simxSetJointTargetPosition(clientID, joint_four_handle, next_theta[3], vrep.simx_opmode_oneshot)
    vrep.simxSetObjectPosition(clientID,sphere_joint4_handle,robot_handle,p_robot[[0,1,2],4],vrep.simx_opmode_blocking)

    vrep.simxSetJointTargetPosition(clientID, joint_five_handle, next_theta[4], vrep.simx_opmode_oneshot)
    vrep.simxSetObjectPosition(clientID,sphere_joint5_handle,robot_handle,p_robot[[0,1,2],5],vrep.simx_opmode_blocking)
    
    vrep.simxSetJointTargetPosition(clientID, joint_six_handle, next_theta[5], vrep.simx_opmode_oneshot)
    vrep.simxSetObjectPosition(clientID,sphere_joint6_handle,robot_handle,p_robot[[0,1,2],6],vrep.simx_opmode_blocking)

    vrep.simxSetObjectPosition(clientID,sphere_end_effector_handle,robot_handle,p_robot[[0,1,2],7],vrep.simx_opmode_blocking)

[[ 0.1        -1.22562556  0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.1         0.06633541  0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.1         1.40178655  0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.1         0.81004128  0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.1         0.90588046  0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.1         0.87950189  0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.        